In [1]:
from pymongo import MongoClient
import pymongo
import pandas as pd
import re
from collections import Counter
import math
import numpy as np
from sshtunnel import SSHTunnelForwarder
import warnings
# 忽略所有警告
warnings.filterwarnings("ignore")

In [2]:
from xgboost import XGBClassifier
model = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor')
print("GPU 設定成功")

GPU 設定成功


In [3]:
#讀取
df = pd.read_excel("data_clearned.xlsx")
df

,Unnamed: 0,縣市,簽約租金,鄉鎮市區,出租人年齡,建物型態,屋齡,幾房,實際使用坪數,是否為弱勢身分,承租人年齡,高齡者,KMeans_生活型態分群,是否電梯大樓
0,0,高雄市,20349,前鎮區,46,透天厝,31,4,56.044175,0,50,0,2,0
1,1,高雄市,20009,前鎮區,49,透天厝,32,4,56.044175,0,52,0,2,0
2,2,高雄市,19808,前鎮區,52,透天厝,36,4,56.044175,0,56,0,2,0
3,3,高雄市,7990,苓雅區,59,公寓,36,4,26.399175,1,65,1,2,0
4,4,高雄市,9931,鳳山區,35,公寓,36,2,29.971700,0,48,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8593,8593,高雄市,17960,三民區,47,住宅大樓（十一層(含)以上有電梯）,29,2,35.177725,0,34,0,2,1
8594,8594,高雄市,13002,前鎮區,65,公寓,36,3,26.822675,1,50,0,2,0
8595,8595,高雄市,16001,楠梓區,49,住宅大樓（十一層(含)以上有電梯）,15,1,21.223400,1,41,0,1,1
8596,8596,高雄市,17997,鳳山區,51,公寓,38,2,22.279125,0,36,0,2,0


In [4]:
print("總樣本數  高齡者  比例")
print(len(df), sum(df['高齡者']), round((sum(df['高齡者']) / len(df)) * 100, 1) if len(df) > 0 else 0)

總樣本數  高齡者  比例
8598 434 5.0


In [5]:
df.columns

Index(['Unnamed: 0', '縣市', '簽約租金', '鄉鎮市區', '出租人年齡', '建物型態', '屋齡', '幾房',
       '實際使用坪數', '是否為弱勢身分', '承租人年齡', '高齡者', 'KMeans_生活型態分群', '是否電梯大樓'],
      dtype='object')

In [6]:
import os
import urllib.request
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rcParams

# 字體下載與註冊：思源黑體繁體
font_url = 'https://github.com/adobe-fonts/source-han-sans/raw/release/OTF/TraditionalChinese/SourceHanSansTC-Regular.otf'
font_path = 'SourceHanSansTC-Regular.otf'

if not os.path.exists(font_path):
    print("正在下載字體...")
    urllib.request.urlretrieve(font_url, font_path)
    print("字體下載完成！")

# 註冊為全域字體（這很重要，能避免 findfont 警告）
fm.fontManager.addfont(font_path)
font_prop = fm.FontProperties(fname=font_path)
rcParams['font.family'] = font_prop.get_name()

# 其餘 import
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import shap
from collections import Counter
from sklearn.preprocessing import StandardScaler

# 特徵與目標變數
SAS_vars = ['屋齡', 'KMeans_生活型態分群', '實際使用坪數', '出租人年齡', '簽約租金', '幾房']
X_XG = df[SAS_vars]
y_XG = df['高齡者']

# 超參數搜尋空間
param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 0.8],
    'gamma': [0, 0.1, 0.3],
    'scale_pos_weight': [1]
}

# 超參數搜尋
xgb_initial = XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42)
random_search = RandomizedSearchCV(
    estimator=xgb_initial,
    param_distributions=param_grid,
    scoring='roc_auc',
    n_iter=100,
    cv=5,
    random_state=42,
    verbose=1,
    n_jobs=-1,
    error_score='raise'
)
random_search.fit(X_XG, y_XG)
best_params = random_search.best_params_
print("\n===== 最佳超參數 =====")
print(best_params)

# 固定最佳參數
xgb_params = best_params.copy()
xgb_params.update({'objective': 'binary:logistic', 'eval_metric': 'logloss', 'random_state': 42})

# 初始化儲存器
xgb_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': [], 'best_threshold': []}
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
all_shap_values = []
all_feature_names = []

# 交叉驗證
for fold, (train_idx, test_idx) in enumerate(kfold.split(X_XG, y_XG)):
    print(f"\n===== 第 {fold + 1} 折交叉驗證 =====")
    X_train_XG, X_test_XG = X_XG.iloc[train_idx], X_XG.iloc[test_idx]
    y_train_XG, y_test_XG = y_XG.iloc[train_idx], y_XG.iloc[test_idx]

    scaler = StandardScaler()
    X_train_XG = pd.DataFrame(scaler.fit_transform(X_train_XG), columns=SAS_vars)
    X_test_XG = pd.DataFrame(scaler.transform(X_test_XG), columns=SAS_vars)

    smote = SMOTE(random_state=42)
    X_train_XG, y_train_XG = smote.fit_resample(X_train_XG, y_train_XG)

    xgb_clf = XGBClassifier(**xgb_params)
    xgb_clf.fit(X_train_XG, y_train_XG)

    # SHAP 分析
    explainer = shap.TreeExplainer(xgb_clf)
    shap_values = explainer(X_test_XG)

    # SHAP summary plot（修改 Y 軸中文字體）
    shap.summary_plot(shap_values, X_test_XG, show=False)
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.tight_layout()
    plt.show()

    all_shap_values.append(np.abs(shap_values.values).mean(axis=0))
    all_feature_names = SAS_vars

    y_pred_proba = xgb_clf.predict_proba(X_test_XG)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test_XG, y_pred_proba)
    best_threshold = thresholds[np.argmax(tpr - fpr)]
    y_pred = (y_pred_proba >= best_threshold).astype(int)

    xgb_metrics['accuracy'].append(accuracy_score(y_test_XG, y_pred))
    xgb_metrics['precision'].append(precision_score(y_test_XG, y_pred))
    xgb_metrics['recall'].append(recall_score(y_test_XG, y_pred))
    xgb_metrics['f1'].append(f1_score(y_test_XG, y_pred))
    xgb_metrics['roc_auc'].append(roc_auc_score(y_test_XG, y_pred_proba))
    xgb_metrics['best_threshold'].append(best_threshold)

    print(f"最佳閥值: {best_threshold:.4f}")
    print(f"準確率: {xgb_metrics['accuracy'][-1]:.4f}, 精確度: {xgb_metrics['precision'][-1]:.4f}, "
          f"召回率: {xgb_metrics['recall'][-1]:.4f}, F1-score: {xgb_metrics['f1'][-1]:.4f}, "
          f"AUC: {xgb_metrics['roc_auc'][-1]:.4f}")

# 統計平均指標
def mean_ci(data):
    mean = np.mean(data)
    sem = np.std(data, ddof=1) / np.sqrt(len(data))
    margin = 1.96 * sem
    return mean, mean - margin, mean + margin

print("\n===== XGBoost 總平均指標與 95% CI =====")
for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
    mean, lower, upper = mean_ci(xgb_metrics[metric])
    print(f"{metric.capitalize():<10} 平均: {mean:.4f} | 95% CI: [{lower:.4f}, {upper:.4f}]")

# SHAP 平均重要性
avg_shap_importance = np.mean(all_shap_values, axis=0)
mean_feature_importance = pd.DataFrame({
    'Feature': all_feature_names,
    'Mean SHAP Importance': avg_shap_importance
}).sort_values(by='Mean SHAP Importance', ascending=False)

print("\n===== 平均特徵重要性 (SHAP across folds) =====")
print(mean_feature_importance)

# 長條圖顯示（套用中文字體）
plt.figure(figsize=(10, 6))
plt.barh(mean_feature_importance['Feature'], mean_feature_importance['Mean SHAP Importance'])
plt.xlabel("SHAP平均特徵重要性", fontproperties=font_prop)
plt.ylabel("特徵名稱", fontproperties=font_prop)
plt.title("SHAP特徵重要性比較（10 折交叉驗證平均）", fontproperties=font_prop)

ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)

plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig("xg_shap_mean_importance_bar.png", dpi=300, bbox_inches="tight")
plt.savefig("xg_shap_mean_importance_bar.svg", bbox_inches="tight")
plt.show()

# 輸出 CSV
mean_feature_importance.to_csv("xg_mean_shap_importance.csv", index=False)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


XGBoostError: [08:04:20] /workspace/src/data/array_interface.cu:44: Check failed: err == cudaGetLastError() (0 vs. 2) : 
Stack trace:
  [bt] (0) /mnt/sda/home/nsysu113/.conda/envs/python39/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x25c1ac) [0x7f824c1fe1ac]
  [bt] (1) /mnt/sda/home/nsysu113/.conda/envs/python39/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(+0xa3f4bc) [0x7f824c9e14bc]
  [bt] (2) /mnt/sda/home/nsysu113/.conda/envs/python39/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x4e3a2e) [0x7f824c485a2e]
  [bt] (3) /mnt/sda/home/nsysu113/.conda/envs/python39/lib/python3.9/site-packages/xgboost/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0xb2) [0x7f824c105522]
  [bt] (4) /mnt/sda/home/nsysu113/.conda/envs/python39/lib/python3.9/lib-dynload/../../libffi.so.8(+0xa052) [0x7f8277731052]
  [bt] (5) /mnt/sda/home/nsysu113/.conda/envs/python39/lib/python3.9/lib-dynload/../../libffi.so.8(+0x8925) [0x7f827772f925]
  [bt] (6) /mnt/sda/home/nsysu113/.conda/en

In [ ]:
# ========= 字體下載與全域設定 =========
import os
import urllib.request
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 字體下載與註冊：思源黑體繁體
font_url = 'https://github.com/adobe-fonts/source-han-sans/raw/release/OTF/TraditionalChinese/SourceHanSansTC-Regular.otf'
font_path = 'SourceHanSansTC-Regular.otf'

if not os.path.exists(font_path):
    print("正在下載字體...")
    urllib.request.urlretrieve(font_url, font_path)
    print("字體下載完成！")

# 註冊字體與設定
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()

# ========= 其餘 import =========
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import shap

# ========= 資料與變數 =========
SAS_vars = ['屋齡', 'KMeans_生活型態分群', '實際使用坪數', '出租人年齡', '簽約租金', '幾房']
X_RF = df[SAS_vars]
y_RF = df['高齡者']

param_grid = {
    'n_estimators': [100, 300, 500, 700],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf_initial = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(
    estimator=rf_initial,
    param_distributions=param_grid,
    scoring='roc_auc',
    n_iter=100,
    cv=5,
    random_state=42,
    verbose=1,
    n_jobs=-1
)
random_search.fit(X_RF, y_RF)

best_params = random_search.best_params_
print("\n===== 最佳超參數 =====")
print(best_params)

rf_params = best_params.copy()
rf_params.update({'random_state': 42})

rf_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': [], 'best_threshold': []}
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
all_shap_values = []
all_feature_names = SAS_vars

for fold, (train_idx, test_idx) in enumerate(kfold.split(X_RF, y_RF)):
    print(f"\n===== 第 {fold + 1} 折交叉驗證 =====")

    X_train_RF, X_test_RF = X_RF.iloc[train_idx], X_RF.iloc[test_idx]
    y_train_RF, y_test_RF = y_RF.iloc[train_idx], y_RF.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_RF)
    X_test_scaled = scaler.transform(X_test_RF)
    X_train_RF = pd.DataFrame(X_train_scaled, columns=SAS_vars)
    X_test_RF = pd.DataFrame(X_test_scaled, columns=SAS_vars)

    smote = SMOTE(random_state=42)
    X_train_RF, y_train_RF = smote.fit_resample(X_train_RF, y_train_RF)

    rf_clf = RandomForestClassifier(**rf_params)
    rf_clf.fit(X_train_RF, y_train_RF)

    explainer = shap.Explainer(rf_clf)
    shap_values = explainer(X_test_RF)

    # SHAP summary plot（中文Y軸處理）
    shap.summary_plot(shap_values[:, :, 1], X_test_RF, show=False)
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.tight_layout()
    plt.show()

    shap_importance = np.abs(shap_values.values[:, :, 1]).mean(axis=0)
    all_shap_values.append(shap_importance)

    y_pred_rf_proba = rf_clf.predict_proba(X_test_RF)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test_RF, y_pred_rf_proba)
    best_threshold = thresholds[np.argmax(tpr - fpr)]
    y_pred_rf = (y_pred_rf_proba >= best_threshold).astype(int)

    rf_metrics['accuracy'].append(accuracy_score(y_test_RF, y_pred_rf))
    rf_metrics['precision'].append(precision_score(y_test_RF, y_pred_rf))
    rf_metrics['recall'].append(recall_score(y_test_RF, y_pred_rf))
    rf_metrics['f1'].append(f1_score(y_test_RF, y_pred_rf))
    rf_metrics['roc_auc'].append(roc_auc_score(y_test_RF, y_pred_rf_proba))
    rf_metrics['best_threshold'].append(best_threshold)

    print(f"最佳閥值: {best_threshold:.4f}")
    print(f"準確率: {rf_metrics['accuracy'][-1]:.4f}, 精確度: {rf_metrics['precision'][-1]:.4f}, "
          f"召回率: {rf_metrics['recall'][-1]:.4f}, F1-score: {rf_metrics['f1'][-1]:.4f}, "
          f"AUC: {rf_metrics['roc_auc'][-1]:.4f}")

def mean_ci(data):
    mean = np.mean(data)
    sem = np.std(data, ddof=1) / np.sqrt(len(data))
    margin = 1.96 * sem
    return mean, mean - margin, mean + margin

print("\n===== Random Forest 平均指標與 95% CI =====")
for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
    mean, lower, upper = mean_ci(rf_metrics[metric])
    print(f"{metric.capitalize():<10} 平均: {mean:.4f} | 95% CI: [{lower:.4f}, {upper:.4f}]")

# SHAP 平均重要性
avg_shap_importance = np.mean(all_shap_values, axis=0)
mean_feature_importance = pd.DataFrame({
    'Feature': all_feature_names,
    'Mean SHAP Importance': avg_shap_importance
}).sort_values(by='Mean SHAP Importance', ascending=False)

print("\n===== 平均特徵重要性 (SHAP across folds) =====")
print(mean_feature_importance)

# 繪製長條圖（含中文支援）
plt.figure(figsize=(10, 6))
plt.barh(mean_feature_importance['Feature'], mean_feature_importance['Mean SHAP Importance'])
plt.xlabel("SHAP平均特徵重要性", fontproperties=font_prop)
plt.ylabel("特徵名稱", fontproperties=font_prop)
plt.title("SHAP特徵重要性比較（10 折交叉驗證平均）", fontproperties=font_prop)

ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)

plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig("rf_shap_mean_importance_bar.png", dpi=300, bbox_inches="tight")
plt.savefig("rf_shap_mean_importance_bar.svg", bbox_inches="tight")
plt.show()

mean_feature_importance.to_csv("rf_mean_rf_shap_importance.csv", index=False)


In [ ]:
# ========= 字體下載與全域設定 =========
import os
import urllib.request
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 字體來源（繁體中文）
font_url = 'https://github.com/adobe-fonts/source-han-sans/raw/release/OTF/TraditionalChinese/SourceHanSansTC-Regular.otf'
font_path = 'SourceHanSansTC-Regular.otf'

if not os.path.exists(font_path):
    print("正在下載字體...")
    urllib.request.urlretrieve(font_url, font_path)
    print("字體下載完成！")

# 註冊字體與設定
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams['axes.unicode_minus'] = False

# ========= 其餘 import =========
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import shap

# ========= 資料與變數 =========
SAS_vars = ['屋齡', 'KMeans_生活型態分群', '實際使用坪數', '出租人年齡', '簽約租金', '幾房']
X_SVM = df[SAS_vars]
y_SVM = df['高齡者']

# ========= SVM 超參數搜尋（使用 Pipeline：scaler→SVC） =========
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(probability=True, random_state=42, cache_size=500))
])

param_distributions = {
    'svc__kernel': ['rbf'],
    'svc__C': [0.1, 0.3, 1, 3, 10, 30, 100],
    'svc__gamma': ['scale', 0.01, 0.03, 0.1, 0.3]  # 對 linear 會被忽略
}

random_search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_distributions,
    scoring='roc_auc',
    n_iter=40,
    cv=5,
    random_state=42,
    verbose=1,
    n_jobs=-1
)
random_search.fit(X_SVM, y_SVM)

best_params = random_search.best_params_
print("\n===== 最佳超參數（Pipeline 命名）=====")
print(best_params)

# 取出 svc 的參數，後續在每折中自行套用（因為我們折內自己做 scaler / SMOTE）
svm_params = {
    'kernel': best_params.get('svc__kernel', 'rbf'),
    'C': best_params.get('svc__C', 1.0),
    'gamma': best_params.get('svc__gamma', 'scale'),
    'probability': True,
    'random_state': 42,
    'cache_size': 500
}

# ========= 10 折交叉驗證 =========
svm_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': [], 'best_threshold': []}
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

all_shap_values = []
all_feature_names = SAS_vars

# SHAP 加速設定
BACKGROUND_K = 20          # kmeans 背景點數
SHAP_NSAMPLES = 100        # Kernel SHAP 採樣數
SHAP_TEST_LIMIT = 500      # 每折最多解釋的測試筆數

for fold, (train_idx, test_idx) in enumerate(kfold.split(X_SVM, y_SVM)):
    print(f"\n===== 第 {fold + 1} 折交叉驗證 =====")

    X_train, X_test = X_SVM.iloc[train_idx], X_SVM.iloc[test_idx]
    y_train, y_test = y_SVM.iloc[train_idx], y_SVM.iloc[test_idx]

    # 標準化
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=SAS_vars, index=X_train.index)
    X_test_scaled  = pd.DataFrame(scaler.transform(X_test), columns=SAS_vars, index=X_test.index)

    # 訓練集 SMOTE（只對訓練集）
    smote = SMOTE(random_state=42)
    X_train_bal, y_train_bal = smote.fit_resample(X_train_scaled, y_train)

    # 訓練 SVM（用最佳參數）
    svm_clf = SVC(**svm_params)
    svm_clf.fit(X_train_bal, y_train_bal)
    
    # ====== SHAP：用正類機率函數 + KernelExplainer（避免多輸出混亂）======
    # 背景點用 kmeans 中心，減少成本
    background = shap.kmeans(X_train_scaled.values, BACKGROUND_K)
    
    # 正類機率函數（回傳一維機率）
    def f_pos(X_np):
        X_df = pd.DataFrame(X_np, columns=SAS_vars)
        return svm_clf.predict_proba(X_df)[:, 1]
    
    # 測試集下采樣（避免太慢）
    if len(X_test_scaled) > SHAP_TEST_LIMIT:
        X_test_for_shap = X_test_scaled.sample(SHAP_TEST_LIMIT, random_state=42)
    else:
        X_test_for_shap = X_test_scaled.copy()
    
    # 用 KernelExplainer（穩定）
    explainer = shap.KernelExplainer(f_pos, background)
    sv = explainer.shap_values(X_test_for_shap.values, nsamples=SHAP_NSAMPLES)
    # sv 形狀: (n_samples, n_features)
    
    # 安全檢查
    assert isinstance(sv, np.ndarray), f"type(sv)={type(sv)}"
    assert sv.ndim == 2 and sv.shape[1] == len(SAS_vars), f"sv shape={sv.shape}, features={len(SAS_vars)}"
    
    # beeswarm / summary_plot（中文 y 軸）
    shap.summary_plot(sv, X_test_for_shap, show=False)
    ax = plt.gca()
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_prop)
    plt.tight_layout()
    plt.show()
    
    # 這一折的特徵平均重要性
    shap_importance = np.mean(np.abs(sv), axis=0)  # (n_features,)
    all_shap_values.append(shap_importance)


    # ===== 評估（Youden’s J 取最佳閾值）=====
    y_pred_proba = svm_clf.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    best_threshold = thresholds[np.argmax(tpr - fpr)]
    y_pred = (y_pred_proba >= best_threshold).astype(int)

    svm_metrics['accuracy'].append(accuracy_score(y_test, y_pred))
    svm_metrics['precision'].append(precision_score(y_test, y_pred, zero_division=0))
    svm_metrics['recall'].append(recall_score(y_test, y_pred, zero_division=0))
    svm_metrics['f1'].append(f1_score(y_test, y_pred, zero_division=0))
    svm_metrics['roc_auc'].append(roc_auc_score(y_test, y_pred_proba))
    svm_metrics['best_threshold'].append(best_threshold)

    print(f"最佳閥值: {best_threshold:.4f}")
    print(f"準確率: {svm_metrics['accuracy'][-1]:.4f}, 精確度: {svm_metrics['precision'][-1]:.4f}, "
          f"召回率: {svm_metrics['recall'][-1]:.4f}, F1-score: {svm_metrics['f1'][-1]:.4f}, "
          f"AUC: {svm_metrics['roc_auc'][-1]:.4f}")

# ========= 統計平均指標與 SHAP 匯出 =========
def mean_ci(data):
    mean = np.mean(data)
    sem  = np.std(data, ddof=1) / np.sqrt(len(data))
    margin = 1.96 * sem
    return mean, mean - margin, mean + margin

print("\n===== SVM 平均指標與 95% CI =====")
for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
    mean, lower, upper = mean_ci(svm_metrics[metric])
    print(f"{metric.capitalize():<10} 平均: {mean:.4f} | 95% CI: [{lower:.4f}, {upper:.4f}]")

# SHAP 平均特徵重要性（跨 10 折）
avg_shap_importance = np.mean(all_shap_values, axis=0)  # (n_features,)
mean_feature_importance = pd.DataFrame({
    'Feature': all_feature_names,
    'Mean SHAP Importance': avg_shap_importance
}).sort_values(by='Mean SHAP Importance', ascending=False)

print("\n===== 平均特徵重要性 (SHAP across folds) =====")
print(mean_feature_importance)

# 繪製長條圖（含中文支援）
plt.figure(figsize=(10, 6))
plt.barh(mean_feature_importance['Feature'], mean_feature_importance['Mean SHAP Importance'])
plt.xlabel("SHAP平均特徵重要性", fontproperties=font_prop)
plt.ylabel("特徵名稱", fontproperties=font_prop)
plt.title("SHAP特徵重要性比較（10 折交叉驗證平均）", fontproperties=font_prop)

ax = plt.gca()
for label in ax.get_yticklabels():
    label.set_fontproperties(font_prop)

plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig("svm_shap_mean_importance_bar.png", dpi=300, bbox_inches="tight")
plt.savefig("svm_shap_mean_importance_bar.svg", bbox_inches="tight")
plt.show()

mean_feature_importance.to_csv("svm_mean_shap_importance.csv", index=False)
